In [1]:

from __future__ import print_function
import argparse
import time
import math
import sys
import warnings
import torch
import torch.nn as nn
import data
import model

In [6]:
device=torch.device("cpu")
ntokens = len(corpus.dictionary)
model = model.RNNModel('LSTM', ntokens, 200, 200,
                           2, embedding_file=None,
                           dropout=0.2, tie_weights=False,
                           freeze_embedding=False).to(device)



def evaluate(data_source):
    """ Evaluate for validation (no adaptation, no complexity output) """
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(20)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, 35):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets.long()).item()
            hidden = repackage_hidden(hidden)
    return total_loss / len(data_source)


def batchify(data, bsz):
    ''' Starting from sequential data, batchify arranges the dataset into columns.
    For instance, with the alphabet as the sequence and batch size 4, we'd get
    a g m s
    b h n t
    c i o u
    d j p v
    e k q w
    f l r x
    These columns are treated as independent by the model, which means that the
    dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
    batch processing.
    '''
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    # Turning the data over to CUDA at this point may lead to more OOM errors
    return data.to(device)


corpus = data.SentenceCorpus('./data','../yue_wiki/files_doc/modified/bpevocab',
                            trainfname='can_train',
                            validfname='can_test')


val_data=batchify(corpus.valid, 20)


evaluate(val_data)

ModuleAttributeError: 'RNNModel' object has no attribute 'RNNModel'

In [21]:
!time python main.py --vocab_file ../yue_wiki/files_doc/modified/bpevocab --data_dir ./data --trainfname can_train --validfname can_test


Libraries loaded
<_io.TextIOWrapper name='./data/can_train' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./data/can_test' mode='r' encoding='UTF-8'>
Traceback (most recent call last):
  File "main.py", line 582, in <module>
    train()
  File "main.py", line 549, in train
    output, hidden = model(data, hidden)
  File "/afs/inf.ed.ac.uk/user/s20/s2059804/miniconda3/envs/msc/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/afs/inf.ed.ac.uk/user/s20/s2059804/msc/neural-complexity/model.py", line 85, in forward
    emb = self.drop(self.encoder(observation))
  File "/afs/inf.ed.ac.uk/user/s20/s2059804/miniconda3/envs/msc/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/afs/inf.ed.ac.uk/user/s20/s2059804/miniconda3/envs/msc/lib/python3.8/site-packages/torch/nn/modules/sparse.py", line 124, in forward
    return F.em